**Coursebook: SQL and Data Analysis Workflow**
- Part 4 of Data Analytics Specialization
- Course Length: 12 hours
- Last Updated: April 2019
___

- Author: [Samuel Chan](https://github.com/onlyphantom)
- Developed by [Algoritma](https://algorit.ma)'s product division and instructors team

# Background

## Top-Down Approach 

The coursebook is part 4 of the **Data Analytics Specialization** offered by [Algoritma](https://algorit.ma). It takes a more accessible approach compared to Algoritma's core educational products, by getting participants to overcome the "how" barrier first, rather than a detailed breakdown of the "why". 

This translates to an overall easier learning curve, one where the reader is prompted to write short snippets of code in frequent intervals, before being offered an explanation on the underlying theoretical frameworks. Instead of mastering the syntactic design of the Python programming language, then moving into data structures, and then the `pandas` library, and then the mathematical details in an imputation algorithm, and its code implementation; we would do the opposite: Implement the imputation, then a succinct explanation of why it works and applicational considerations (what to look out for, what are assumptions it made, when _not_ to use it etc).

## Learn-by-Building

This coursebook is intended for participants who have completed the preceding courses offered in the **Data Analytics Developer Specialization**. This is the third course, **SQL and Data Visualization with Pandas**.

The coursebook focuses on:

- Querying from SQL Databases
- SQL Joins
- SQL Conditional Statement
- End to end data analysis

At the end of this course is a Graded Asssignment section, where you are expected to apply all that you've learned on a new dataset, and attempt the given questions.

# Working with SQL Databases

There are a great number of python modules that provide functionalities to work with databases of all variants and flavors. For a MySQL database, we may form a connection using `pymysql` or one of many other alternatives:

```
import pymysql
conn = pymysql.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    db=database)
```

We can then use `pd.read_sql_query()`, passing in the connection:
```
sales = pd.read_sql_query("SELECT * FROM sales", conn)
```

Under the hood, `pandas` uses SQLAlchemy so any database supported by that library will work. This isn't something you need to worry about at this stage of your learning journey, but for the sake for practice, let's also see how a connection URI for a SQLite database looks like:

In [3]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("data_input/chinook.db")

albums = pd.read_sql_query("SELECT * FROM albums", conn)
albums.head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


In the above command, we asked for all columns of a table to be returned to us through the `SELECT *` command. Well, columns of which table? That would be `tables`. Together they form an SQL query:

`SELECT * FROM albums`

The database we're working with have a few tables populated with sample data. The database has the following schema:
![](assets/chinookschema.png)

#### Knowledge Check

We'll create a `DataFrame`: this time select all columns from the `artists` table. Recall that when we use `pd.read_sql_query()` command we pass in the SQL query as a string, and add a connection as the second parameter. Save the output as a `DataFrame`.

Your DataFrame should be constructed like this:

`__ = pd.read_sql_query("SELECT __ FROM __ ", conn)`

Question:
1. How many rows are there in your DataFrame?

In [13]:
## Your code below


## -- Solution code

The `pd.read_sql_query` is most commonly used with that two parameters above, but on its [official documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql_query.html) is a list of other parameters we can use as well. 

In the following cell, we use a similar SQL query with an additional `LIMIT` statement to limit the output to the first 5 records (rows). However, notice that we also set `index_col` so the specified column is recognized as the index:

In [19]:
pd.read_sql_query("SELECT * FROM artists LIMIT 5", 
                  conn, 
                  index_col='ArtistId')

,Name
ArtistId,
1,AC/DC
2,Accept
3,Aerosmith
4,Alanis Morissette
5,Alice In Chains


## SQL Joins

`JOIN` statements are used to combine records from two tables. We can have as many `JOIN` operations as we want in a SQL query.

Below is a diagram of the different types of SQL `JOIN` operations:

![](assets/sqljoins.png)

Credit: Data & Object Factory, LLC

In most business scenarios though, a `LEFT JOIN` is almost always the type of `JOIN` you want - it is very direct (and therefore easy to reason about). Left join return all records in the left table regardless if any of those records have a match in the right table.

The `INNER JOIN` is also very intuitive and easily understood. This query return all of the records in the left table that has a matching record in the right table.

> As a personal side note, I've worked at companies where `RIGHT JOIN` is outright forbidden in favor of `LEFT JOIN`: directness and ease-of-understanding aside, all right joins can be replaced by the opposite left join. 
> 
> The `OUTER JOIN` (also referred to as `FULL JOIN`) is also quite uncommon in practice. Performance reason aside, an outer join return all of the records from both tables regardless if there is a match or not, resulting in a DataFrame that has potentially a lot of `NULL` values.

Consider the database schema illustraation again and pay attention to two tables and their respective columns:

1. `albums`: 
    - `AlbumId`, `Title`, `ArtistId`

2. `artists`: `
    - `ArtistId`, `Name`    

We want a `pandas` DataFrame containing the `AlbumId`, `Title` and `Name`. Notice that `Name` is from the `artists` table while the other columns are from the `albums` table. What is a reasonable strategy?

The most straightforward solution is the `LEFT JOIN`, let's see an example:

In [123]:
albums = pd.read_sql_query("SELECT AlbumId, Title, a.Name \
                           FROM albums \
                           LEFT JOIN artists as a \
                           ON a.ArtistId = albums.ArtistId", conn)
albums.head()

,AlbumId,Title,Name
0,1,For Those About To Rock We Salute You,AC/DC
1,2,Balls to the Wall,Accept
2,3,Restless and Wild,Accept
3,4,Let There Be Rock,AC/DC
4,5,Big Ones,Aerosmith


Notice that in the code above, we place a backslash (`\`) character so we have line continuation and the newline will be ignored. This allows SQL to treat the entire query string as if they were essentially one line.

In [35]:
pd.read_sql_query("SELECT * FROM albums", conn).head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


#### Knowledge Check

Consider the database schema illustraation again and pay attention to two tables and their respective columns:

1. `albums`: `AlbumId`, `Title`, `ArtistId`

2. `tracks`: `TrackId`, `Name`, `AlbumId`, `GenreId`, ... `UnitPrice` 
    
3. `genres`: `GenreId`, `Name`

Create a `DataFrame` containing all columns from the `tracks` table; Additionally, it should also contain:
    - The `Title` column from the `albums` table
    - The `Name` column from the `artists` table
    - The `Name` column from the `genres` table 

> **Hint 1**: In your `SELECT` statement, you can use `SELECT tracks.* FROM TRACKS` to select all columns from the `TRACKS` table
> 
> **Hint 2**: When we write `SELECT tracks.Name as tracksName`, we are renaming the output column from `Name` to `tracksName` using a technique called column aliasing. You may optionally consider doing this for columns that share the same name across different tables 

Set the `TrackId`column to be the index. The resulting `DataFrame` should has 11 columns.

Give your `DataFrame` a name: name it `tracks`. Perform EDA on `tracks` to answer the following question:

1. Use `tail()` to inspect the last 5 rows of data. Which genre is present in the last 5 rows of our `tracks` DataFrame (Check all that apply)?
    - [ ] Latin
    - [ ] Classical
    - [ ] Soundtrack
    - [ ] Pop

2. Apply `pd.crosstab(..., columns='count')`, `.value_counts()`, or any other techniques you've learned to compute the frequency table of Genres in your DataFrame. Which is among the top 3 most represented genres in the `tracks` DataFrame?
    - [ ] Latin
    - [ ] Classical
    - [ ] Soundtrack
    - [ ] Pop

3. Use `groupby()` on Artist Name and compute the `mean()` on the `UnitPrice` of each tracks. You will realize that most artists price their tracks at 0.99 (`mean`) but there are several artists where the `mean()` is 1.99. Which of the Artist has a mean of 0.99 `UnitPrice`:
    - [ ] The Office
    - [ ] Aquaman
    - [ ] Pearl Jam
    - [ ] Lost

In [25]:
## Your code below


## -- Solution code

## WHERE statements

We've seen how to use do some of the most common SQL operations this far. In particular, we have:

- Learned how to write `SELECT` statements  
- Use `index_col` in the `pd.read_sql_query()` method  
- SQL Join operations
- Use SQL Aliases

In the following example, we'll look at one more technique in the SQL arsenal: the `WHERE` clause

A `WHERE` clause is followed by a **condition**. If we want to query for all invoices where country of the billing address is Germany, we can add a `Where` clause to our sql query string:

In [98]:
germany = pd.read_sql_query("SELECT * FROM invoices WHERE BillingCountry = 'Germany'", conn)
germany.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,6,37,2009-01-19 00:00:00,Berger Straße 10,Frankfurt,None,Germany,60316,0.99
2,7,38,2009-02-01 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
3,12,2,2009-02-11 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,13.86
4,29,36,2009-05-05 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,1.98


`WHERE` conditions can be combined with `AND`, `OR` and `NOT`. Supposed we want to create a DataFrame containing all invoices where the billing country is **not** Indonesia, we can do the following:

In [105]:
international = pd.read_sql_query("SELECT * FROM invoices WHERE NOT BillingCountry = 'Indonesia'", conn)
international.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86


#### Knowledge Check

Edit the following code to include a `WHERE` clause. We want the returned DataFrame to contain only the `Pop` genre and only when the `UnitPrice` of the track is 0.99:

```
popmusic = pd.read_sql_query("SELECT tracks.*, genres.Name as GenreName \
                            FROM tracks \
                            LEFT JOIN genres ON _____ \
                            WHERE genres.Name = ____ AND _____, 
                           conn,
                           index_col='TrackId'

)
```

Question:
1. How many rows are there in `popmusic`?

In [106]:
## Your code below


## -- Solution code

### Using `LIKE` Operator

Using a `WHERE` statement in a query can be beneficial to pull relevant data for our analysis. A common operator for a WHERE statement is `LIKE`.
Consider the following SQL Query:

In [9]:
germany = pd.read_sql_query("SELECT * FROM invoices \
                             WHERE BillingCountry = 'Germany' \
                             AND BillingPostalCode LIKE '107%'", conn)
germany.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,7,38,2009-02-01 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
1,29,36,2009-05-05 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,1.98
2,30,38,2009-05-06 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,3.96
3,40,36,2009-06-15 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,13.86
4,52,38,2009-08-08 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,5.94


A `LIKE` operator is a can be used to match a certain part of the data and can be really useful if we need to perform a partial matching to a specific value rather than using an equal-to operator. The `107%` you see in the query means to extract value in `BillingPostalCode` that starts with the number 107. This is really helpful when you are aiming to extract data specifically on specific region. In Germany, you would know that Wilmersdorf and Tempelhof in Berlin has postal code starting with 107.

**Discussion:**

If you were to put `%` on the end, you are matching everything that starts with 107, if you put it on the start `%107`, means you are matching everything that ends in 107. What do you think will came up if you use `%`  before and after the pattern match?

#### Dive Deeper:

Let's take a look at another case, consider the following data frame:

In [35]:
customerinv = pd.read_sql_query("SELECT firstname, lastname, email, company, \
                                 invoiceid, invoicedate, billingcountry, total \
                                 FROM invoices \
                                 left join customers \
                                 on invoices.customerId = customers.customerId", conn)
customerinv.head()

,FirstName,LastName,Email,Company,InvoiceId,InvoiceDate,BillingCountry,Total
0,Leonie,Köhler,leonekohler@surfeu.de,None,1,2009-01-01 00:00:00,Germany,1.98
1,Bjørn,Hansen,bjorn.hansen@yahoo.no,None,2,2009-01-02 00:00:00,Norway,3.96
2,Daan,Peeters,daan_peeters@apple.be,None,3,2009-01-03 00:00:00,Belgium,5.94
3,Mark,Philips,mphilips12@shaw.ca,Telus,4,2009-01-06 00:00:00,Canada,8.91
4,John,Gordon,johngordon22@yahoo.com,None,5,2009-01-11 00:00:00,USA,13.86


Within the first 6 data, you could see the `Company` column may not be reliable since most of them are filled in with None. Some people might find it unnecessary to fill in the information. But if you pay close attention to the `Email` column, you could see some people have an email domain at `apple`, that could be an indicator of their company.

1. How would your conditional `WHERE` statement would be like if you want to count the number of customers that are working at Apple Inc.?

    Try to complete the following codes:

```
applecust = pd.read_sql_query("SELECT firstname, lastname, email, company, \
                               invoiceid, invoicedate, billingcountry, total \
                               FROM invoices \
                               left join customers \
                               on invoices.customerId = customers.customerId \
                               WHERE email like '___'", conn)

)
```

In [1]:
## Your code below


## -- Solution code

Based on the data queried, how many of the customers is working at Apple Inc.?

- [ ] 412  
- [ ] 49   
- [ ] 7  
- [ ] 14  

# Learn-by-Building

## Graded Assignment

The following learn-by-building exercise will guide you through the process of building out a simple analysis along with some accompanying charts. This module is considerably more difficult than similar exercise blocks in the past, but it sure is a lot more rewarding!

Let's try by first constructing a DataFrame using the `read_sql_query()` method that we've grown familiar to. We want to develop a simple sales visualization report of our top 5 key markets (`Country` column in `customers`) ranked by Sales (`Total` column in `invoices`). 

We also want to identify our top 5 customers by name (`FirstName`, `LastName`) in the report.

Last but not least, we want the report to include a day-of-week analysis on sales performance, and for that we will need the `InvoiceDate` column. 

> **Hint 1**: `pandas` has built-in methods of extracting the name of day in a week. We've seen this in Part 2 of this specialization (**Working with Datetime chapter**). An example usage is:
>
> `x['InvoiceDOW'] = x['InvoiceDate'].dt.weekday_name`
>
>  **Hint 2**: In `read_sql_query`, you can use the `parse_dates='InvoiceDate'` argument to have the specified column parsed as date, saving you from a `to_datetime()` conversion

In [195]:
## Your code below


## -- Solution code

1. Which of the following customers are among the top 5 customers ranked by their respective total (`Total`)? Choose all that apply.
    - [ ] Victor Stevens
    - [ ] Hugh O'Reilly
    - [ ] Puja Srivastava
    - [ ] Ellie Sullivan

2. Which of the following countries are among the top 5 grossing ranked by their respective total (`Total`)? Choose all that apply.
    - [ ] United Kingdom
    - [ ] Germany
    - [ ] Portugal
    - [ ] India
    
3. How many sales transactions (or records, or rows) were from the top 5 countries?
    - Answer:

In [245]:
## Your code below


## -- Solution code

Use the code block below as reference to create a DataFrame containing records where the customers are from one of the top 5 countries. The number of rows in this new DataFrame should be equal to your answer in question (3). Notice that we have additionally converted our day-of-week column into a `Categorical` column:

```
top5 = x.groupby('Country').Total.sum().sort_values(ascending=False).head().index.to_list()
invo_topcountry = invo[invo['Country'].isin(top5)].copy()
# Represents a categorical variable in classic R fashion (pandas documentation) and have it ordered
invo_topcountry['InvoiceDOW'] = pd.Categorical(invo_topcountry.InvoiceDOW, categories=['Monday', 'Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday'], ordered=True)
invo_topcountry.shape # same as answer for question (3)
```

4. Using the `pd.pivot_table()` method, answer the next two questions
    - 4.1 How much sales did we make in total (`Total`) from the France market on a Tuesday?
    - 4.2 How much sales did we make in total (`Total`) from all the top 5 markets combined on a Friday?

5. Give the `pivot_table` that you created a name. Remove the margins if they were included in the earlier step. Create a visualization by chaining the `.plot()` method on the pivot table. Set the `kind` parameter to be one of:
    - `line`
    - `bar`
    - `hist`
    - `box`
    
Which are the most appropriate plot type(s)? Choose THREE:
    - [ ] `hist`
    - [ ] `box`
    - [ ] `bar`
    - [ ] `scatter`
    - [ ] `line`

In [257]:
## Your code below


## -- Solution code

#### Reference Answer

Reference code for Learn-By-Building:
```
tracks = pd.read_sql_query("SELECT tracks.*, albums.Title, artists.Name as ArtistName, genres.Name as GenreName \
                            FROM tracks \
                            LEFT JOIN albums ON albums.AlbumId = tracks.AlbumId \
                            LEFT JOIN artists ON artists.ArtistId = albums.ArtistId \
                            LEFT JOIN genres ON genres.GenreId=tracks.GenreId", 
                           conn,
                           index_col='TrackId'

)

tracks.shape
```